In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import pipeline
from sklearn.metrics import mean_squared_error as MSE

# Pandas Profiling package
from pandas_profiling import ProfileReport

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Dropout, Flatten
from keras.layers.convolutional import Conv1D, Conv2D

In [ ]:
df_train = pd.read_csv("../input/segmind-grand-ai-challenge-2021/TRAIN.csv")
df_store = pd.read_csv("../input/segmind-grand-ai-challenge-2021/store.csv")

### **Pandas Profiling**

In [ ]:
train_profile = ProfileReport(
    df_train, title="Training Dataset", html={"style": {"full_width": True}}, sort="None"
)

In [ ]:
train_profile

In [ ]:
store_profile = ProfileReport(
    df_store, title="Test Dataset", html={"style": {"full_width": True}}, sort="None"
)

### **EDA on train.csv**

In [ ]:
df_train.head()
#print(df_train.shape)

In [ ]:
df_train.info()

In [ ]:
df_train["Store"].value_counts()

In [ ]:
plt.figure(figsize = (12, 4))
sns.countplot(x = 'DayOfWeek', data = df_train, palette='rainbow')    #df_train['DayOfWeek'].value_counts()
plt.title("Distribution among Day of weeks")
plt.show()

In [ ]:
df_train["Date"] = pd.to_datetime(df_train["Date"])
df_train.head()

In [ ]:
train_gp = df_train.groupby('Store')

In [ ]:
idx = 1
store = train_gp.get_group(idx)
store.sort_values(by = 'Date', inplace = True)
store.head()

In [ ]:
st = 6
plt.figure(figsize = (20, 4))
plt.plot(df_train[df_train['Store'] == st]["Date"], df_train[df_train['Store'] == st]["Sales"])
plt.show()

In [ ]:
plt.figure(figsize = (15, 5))

plt.subplot(121)
sns.countplot(x = 'Open', data = df_train, palette='rainbow')    #df_train['DayOfWeek'].value_counts()

plt.subplot(122)
sns.countplot(x = 'Open', hue = 'DayOfWeek', data = df_train, palette = "Set3")

plt.show()

In [ ]:
plt.figure(figsize = (15, 5))

plt.subplot(121)
sns.countplot(x = 'Promo', data = df_train, palette='rainbow')    #df_train['DayOfWeek'].value_counts()

plt.subplot(122)
sns.countplot(x = 'Promo', hue = 'DayOfWeek', data = df_train, palette = "Set3")

plt.show()

In [ ]:
df_train['StateHoliday'][df_train['StateHoliday'] == 0] = '0'
df_train['StateHoliday'].value_counts()

In [ ]:
state_holiday_encoder = LabelEncoder()
df_train['StateHoliday'] = state_holiday_encoder.fit_transform(df_train['StateHoliday'])

In [ ]:
state_holiday_encoder.classes_

In [ ]:
plt.figure(figsize = (15, 5))

plt.subplot(121)
sns.countplot(x = 'StateHoliday', data = df_train, palette='rainbow')   

plt.subplot(122)
sns.countplot(x = 'StateHoliday', hue = 'DayOfWeek', data = df_train, palette = "RdGy")

plt.show()

In [ ]:
plt.figure(figsize = (15, 5))

plt.subplot(121)
sns.countplot(x = 'SchoolHoliday', data = df_train, palette='rainbow')   

plt.subplot(122)
sns.countplot(x = 'SchoolHoliday', hue = 'DayOfWeek', data = df_train, palette = "Set3")

plt.show()

In [ ]:
plt.figure(figsize = (8, 6))
sns.heatmap(df_train.corr(), annot=True, cmap="RdYlGn", fmt = "0.4f")
plt.show()

### **EDA on store.csv**

In [ ]:
df_store.head()
#print(df_store.shape)

In [ ]:
df_store['Store'].value_counts()

In [ ]:
df_store.info()

In [ ]:
df_store['StoreType'].value_counts()

In [ ]:
df_store['Assortment'].value_counts()

In [ ]:
df_store['Promo2'].value_counts()

### **Data Pre-processing**

In [ ]:
df = pd.merge(df_train, df_store, how = 'left', left_on = 'Store', right_on = 'Store')
df.head()

In [ ]:
input_cols = ['Store', 'DayOfWeek', 'Date', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'Promo2', 
             'CompetitionDistance']
target_cols = ['Sales']
df_new = df[input_cols + target_cols]
df_new.head()

In [ ]:
# Now, I am using DateTime so this will not work
#df_new['Year'] = df_new['Date'].apply(lambda x: int(x.split('-')[0]))
#df_new['Month'] = df_new['Date'].apply(lambda x: int(x.split('-')[1]))
#df_new['Day'] = df_new['Date'].apply(lambda x: int(x.split('-')[2]))

In [ ]:
#df_new.drop(columns='Date', inplace = True)
df_new.head()

In [ ]:
store_type_encoder = LabelEncoder()
assortment = LabelEncoder()

df_new['StoreType'] = store_type_encoder.fit_transform(df_new['StoreType'])
df_new['Assortment'] = assortment.fit_transform(df_new['Assortment'])

In [ ]:
df_new.head()

In [ ]:
print(len(df_new[df_new['CompetitionDistance'].isnull()]))
df_new = df_new.dropna()

print(len(df_new))

In [ ]:
plt.figure(figsize = (14, 8))
sns.heatmap(df_new.corr(), annot=True, cmap="YlGnBu", fmt = "0.4f")
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(df_new.drop(columns = ['Sales', 'Date']))

In [ ]:
X.shape

### **Split data into train and test data**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, np.asarray(df_new['Sales']), test_size= 0.3) 

print(x_train.shape)
print(x_test.shape)

### **Decision Tree Regressor**

In [ ]:
from sklearn.tree import DecisionTreeRegressor
Regressor = DecisionTreeRegressor(criterion='mse')

In [ ]:
Regressor.fit(x_train, y_train)

In [ ]:
Regressor.score(x_test, y_test)

In [ ]:
pred_dt = Regressor.predict(x_test)
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred_dt))
print("RMSE : % f" %(rmse))

In [ ]:
sns.displot(y_test - pred_dt)
plt.show()

In [ ]:
plt.scatter(y_test, pred_dt, alpha = 0.5)
plt.xlabel("y_test")
plt.ylabel("y_pred")
plt.show()

### **RandomForest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(random_state=0, verbose=1, n_jobs=5)

In [ ]:
#pd.DataFrame(x_train).describe()

In [ ]:
#regr.fit(x_train, y_train)
regr.fit(X, df_new['Sales'].values)

In [ ]:
regr.score(x_test, y_test)

In [ ]:
pred_rf = regr.predict(x_test)

# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred_rf))
print("RMSE : % f" %(rmse))

### **XGBoost**

In [ ]:
import xgboost as xg

In [ ]:
xgb_r = xg.XGBRegressor(objective ='reg:linear',
                  n_estimators = 50, seed = 123)

In [ ]:
xgb_r.fit(x_train, y_train, verbose=1)

In [ ]:
# Predict the model
pred_xgbr = xgb_r.predict(x_test)
  
# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred_xgbr))
print("RMSE : % f" %(rmse))

### **Neural Network**

In [ ]:
from tensorflow import keras
from keras import layers

In [ ]:
model = keras.Sequential()

model.add(layers.Dense(26, activation = 'relu', input_shape = (x_train.shape[1], ))),
model.add(layers.BatchNormalization()),
model.add(layers.Dense(45, activation = 'relu')),
model.add(layers.BatchNormalization()),
model.add(layers.Dropout(0.2)),
model.add(layers.Dense(100, activation = 'relu')),
model.add(layers.BatchNormalization()),
model.add(layers.Dropout(0.2)),
model.add(layers.Dense(50, activation = 'relu')),
model.add(layers.BatchNormalization()),
model.add(layers.Dropout(0.2)),
model.add(layers.Dense(20, activation = 'relu')),
model.add(layers.BatchNormalization()),
model.add(layers.Dropout(0.2)),
model.add(layers.Dense(10, activation = 'relu')),
model.add(layers.Dense(1, activation = 'linear')),

model.summary()

In [ ]:
model.compile(loss = 'mse',
              optimizer = 'adam',
             metrics = ['mse'])

history = model.fit(X, df_new['Sales'], verbose= 1, epochs = 10, batch_size = 256, 
                    validation_split = 0.2, shuffle= True, use_multiprocessing= True)

In [ ]:
pred_rf = model.predict(x_test)

# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred_rf))
print("RMSE : % f" %(rmse))

### **AutoML**

Let's see the power of autoML

In [ ]:
import h2o
from h2o.automl import H2OAutoML

In [ ]:
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes

)

In [ ]:
train = h2o.H2OFrame(df_new)
#test = h2o.H2OFrame(x_test)

In [ ]:
x = train.columns
y = "Sales"
x.remove(y)

In [ ]:
aml = H2OAutoML(max_models=2, seed=1)
aml.train(x=x, y=y, training_frame=train)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

### **LSTM**

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(units = 64, input_shape = (None, x_train.shape[1])))
model_lstm.add(Dropout(0.4))
model_lstm.add(Dense(1))

model_lstm.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
model_lstm.summary()

In [ ]:
plot_model(model_lstm)

In [ ]:
x_train = X
y_train = np.asarray(df_new['Sales']).reshape((len(df_new), 1))

print(x_train.shape, y_train.shape)

In [ ]:
#history_lstm = model_lstm.fit(x_train,y_train,batch_size = 4096,epochs = 10)

### **Model Comparision**

In [ ]:
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge, HuberRegressor, LogisticRegression, BayesianRidge
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
models = [['LinearRegression : ', LinearRegression()],
          ['Lasso : ', Lasso()],
          ['Ridge : ', Ridge()],
          ['KNeighborsRegressor : ', KNeighborsRegressor()],
          ['DecisionTreeRegressor : ', DecisionTreeRegressor()],
          ['RandomForestRegressor : ', RandomForestRegressor()],
          ['AdaBoostRegressor : ', AdaBoostRegressor()],
          ['GradientBoostingRegressor : ', GradientBoostingRegressor()],
          ['ExtraTreeRegressor : ', ExtraTreeRegressor()],
          ['HuberRegressor : ', HuberRegressor()],
          ['XGBRegressor : ', XGBRegressor()],
          ['BayesianRidge : ', BayesianRidge()]]

In [ ]:
for name, model in models:
    model=model
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    print(name, (np.sqrt(mean_squared_error(y_test, predictions))))

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV

In [ ]:
algorithms = {
    'XGBRegressor' : {
        'model' : XGBRegressor(),
        'param' : {
            'learning_rate' : [0.5, 0.8, 0.1, 0.20, 0.25, 0.30],
            'max_depth' : [3, 5, 7, 9, 11, 13, 15],
            'gamma' : [0.1,0.2, 0.3, 0.4, 0.5],
            'min_child_weight' : [1, 3, 5, 7, 9],
            'colsample_bytree' : [0.5, 0.8, 0.1, 0.20, 0.25, 0.30]
        }
    },
    'RandomForestRegressor' : {
        'model' : RandomForestRegressor(),
        'param' : {
            'n_estimators' : [300, 500, 700, 1000, 2100],
            'max_depth' : [3, 5, 7, 9, 11, 13, 15],
            'max_features' : ["auto", "sqrt", "log2"],
            'min_samples_split' : [2, 4, 6, 8]
        }
    },
    'GradientBoostingRegressor' : {
        'model' : GradientBoostingRegressor(),
        'param' : {
            'learning_rate' : [0.5, 0.8, 0.1, 0.20, 0.25, 0.30],
            'n_estimators' : [300, 500, 700, 1000, 2100],
            'criterion' : ['friedman_mse', 'mse']
        }
    }
}

In [ ]:
score = []

for name, mp in algorithms.items() :
    rs = RandomizedSearchCV(estimator = mp['model'], param_distributions = mp['param'], cv = 10, n_jobs=-1, verbose=3)
    rs.fit(x_train, y_train)
    score.append({
        'model': name,
        'score' : rs.best_score_,
        'params' : rs.best_params_
    })

In [ ]:
score

### **Submission File**

In [ ]:
df_test = pd.read_csv("../input/segmind-grand-ai-challenge-2021/TEST.csv")
df_test.head()

In [ ]:
test_df = pd.merge(df_test, df_store, how = 'left', left_on = 'Store', right_on = 'Store')
test_df.head()

In [ ]:
test_df = test_df[input_cols]
test_df.head()

In [ ]:
test_df['Year'] = test_df['Date'].apply(lambda x: int(x.split('-')[0]))
test_df['Month'] = test_df['Date'].apply(lambda x: int(x.split('-')[1]))
test_df['Day'] = test_df['Date'].apply(lambda x: int(x.split('-')[2]))
test_df.drop(columns='Date', inplace = True)

test_df.head()

In [ ]:
test_df['StoreType'] = store_type_encoder.transform(test_df['StoreType'])
test_df['Assortment'] = assortment.transform(test_df['Assortment'])
test_df['StateHoliday'] = state_holiday_encoder.transform(test_df['StateHoliday'])
test_df['CompetitionDistance'].fillna(test_df['CompetitionDistance'].mean(), inplace = True)

test_X = scaler.transform(test_df)

In [ ]:
test_df.describe()

In [ ]:
pred = regr.predict(test_X)
#pred

In [ ]:
sub1 = pd.DataFrame(pred)
sub1 = sub1.reset_index()

In [ ]:
#sub1['Index'] = sub1['Index'] - 1
sub1 = sub1.rename(columns={"index": "Index", 0: "Sales"})
sub1.head()

sub1.to_csv('sub_v4.csv', header = True, index=False)

In [ ]:
sub_979 = pd.read_csv("./sub_v4.csv")
sub_979.head()

In [ ]:
print("MSE: ", (((sub_979['Sales'] - sub1['Sales']))))